In [200]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True

import numpy as np
import pandas as pd
import time as time
from time import sleep
import random


In [165]:
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.library.parameters import SeasonAll

In [166]:
seasons = ['20{}-{}'.format(x, x+1) for x in range(10, 21)]
dates = np.array([])
for season in seasons:
    games = leaguegamelog.LeagueGameLog(season=season).get_data_frames()[0]
    dates = np.append(dates, games['GAME_DATE'].unique())

dates

array(['2010-10-26', '2010-10-27', '2010-10-28', ..., '2021-01-23',
       '2021-01-24', '2021-01-25'], dtype=object)

In [159]:
dates = dates.tolist()

In [160]:
dates[-10:]


['2021-01-16',
 '2021-01-17',
 '2021-01-18',
 '2021-01-19',
 '2021-01-20',
 '2021-01-21',
 '2021-01-22',
 '2021-01-23',
 '2021-01-24',
 '2021-01-25']

In [186]:
for date in dates[-10:]:
    print(date)

    # Points Spread Data
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/pointspread/?date={}'.format(date)
    path = './chromedriver.exe'

    driver = webdriver.Chrome(path, options=options)

    driver.get(web)

    gm_date = []
    away = []
    home = []
    away_scoreboard = []
    home_scoreboard = []
    away_spread = []
    home_spread = []
    away_moneyline = []
    home_moneyline = []
    total = []

    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')
   
    for event in single_row_events:
        away.append(event.find_elements_by_class_name('participantBox-3ar9Y')[0].text)
        home.append(event.find_elements_by_class_name('participantBox-3ar9Y')[1].text)
        gm_date.append(date)

        away_scores = []
        home_scores = []

        scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')
        for score in scoreboard:
            quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
            for i in range(len(quarters)):
                scores = quarters[i].text.split('\n')
                away_scores.append(int(scores[0]))
                home_scores.append(int(scores[1]))
            away_scoreboard.append(away_scores)
            home_scoreboard.append(home_scores)

        spreads = event.find_elements_by_class_name('oddsContainer-15U4I')
        for s in spreads:
            away_spread.append(s.text.split('\n')[0])
            home_spread.append(s.text.split('\n')[1])


    if len(away) != len(away_scoreboard):
        games_to_remove = len(away) - len(away_scoreboard)
        away = away[:-games_to_remove]
        home = home[:-games_to_remove]
        away_spread = away_spread[:-games_to_remove]
        home_spread = home_spread[:-games_to_remove]

    sleep(random.randint(3,7))


2021-01-16
2021-01-16
2021-01-16
2021-01-16
2021-01-16
2021-01-16
2021-01-16
2021-01-17
2021-01-17
2021-01-17
2021-01-17
2021-01-17
2021-01-17
2021-01-17
2021-01-18
2021-01-18
2021-01-18
2021-01-18
2021-01-18
2021-01-18
2021-01-18
2021-01-18
2021-01-18
2021-01-19
2021-01-19
2021-01-20
2021-01-20
2021-01-20
2021-01-20
2021-01-20
2021-01-20
2021-01-20
2021-01-20
2021-01-20
2021-01-20
2021-01-21
2021-01-21
2021-01-21
2021-01-22
2021-01-22
2021-01-22
2021-01-22
2021-01-22
2021-01-22
2021-01-22
2021-01-22
2021-01-22
2021-01-22
2021-01-22
2021-01-22
2021-01-22
2021-01-23


IndexError: list index out of range

In [206]:
web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/pointspread/?date=2021-01-23'
path = './chromedriver.exe'

driver = webdriver.Chrome(path, options=options)

single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

away_spread = []
home_spread = []

for event in single_row_events:
    print(event)
    spreads = event.find_elements_by_class_name('adjust-1uDgI')
    print(len(spreads))
    print(spreads)
#         away_spread.append(s.text.split('\n')[0])
#         home_spread.append(s.text.split('\n')[1])
        
        
# print(away_spread, home_spread)


In [183]:
pd.DataFrame({'away_team':away,
             'home_team':home,
             'away_scoreboard':away_scoreboard,
             'home_scoreboard':home_scoreboard,
             'away_spread':away_spread,
             'home_spread':home_spread})

,away_team,home_team,away_scoreboard,home_scoreboard,away_spread,home_spread
0,Houston,San Antonio,"[30, 23, 19, 19, 91]","[21, 29, 25, 28, 103]",+7-110,-7-110
1,Orlando,Brooklyn,"[26, 24, 33, 32, 115]","[29, 23, 32, 38, 122]",+9½-110,-9½-110
2,Charlotte,Toronto,"[27, 37, 23, 26, 113]","[31, 35, 22, 28, 116]",+6½-110,-6½-110
3,Philadelphia,Memphis,"[26, 28, 22, 28, 104]","[25, 35, 28, 18, 106]",+2-105,-2-115
4,Detroit,Miami,"[29, 23, 38, 30, 120]","[23, 33, 19, 25, 100]",+5½-110,-5½-110
5,Atlanta,Portland,"[38, 28, 20, 20, 106]","[34, 25, 27, 26, 112]",+5-105,-5-115


In [ ]:
df

In [162]:
    # Get totals
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/totals/?date={}'.format(date)
    path = './chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(web)

    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')
    for event in single_row_events:
        t = event.find_element_by_class_name('oddsContainer-15U4I')
        total.append(t.text.split('-')[0])


    # Get Moneylines
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date={}'.format(date)
    path = './chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(web)

    away_moneyline = []
    home_moneyline = []

    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')
    for event in single_row_events:
        ml = event.find_element_by_class_name('oddsContainer-15U4I')
        away_moneyline.append(ml.text.split('\n')[0])
        home_moneyline.append(ml.text.split('\n')[1])


    sleep(random.randint(3,7))

df = pd.DataFrame({'away_team':away,
                  'home_team':home,
                  'away_scoreboard':away_scoreboard,
                  'home_scoreboard':home_scoreboard,
                  'away_spread':away_spread,
                  'home_spread':home_spread,
                  'away_moneyline':away_moneyline,
                  'home_moneyline':home_moneyline,
                  'total':total})

['-450', '+270', '-400', '+250', '-500', '+250', '-140'] ['+325', '-340', '+300', '-300', '+350', '-300', '+120']
